**Author:**
* [Daniela Souza de Oliveira](https://github.com/Danielaso)

*created in jul/2020*

# Comparison IAC vs. VCP

**Inputs:** IAC_1890_2018.csv, VCP

*   VCP climatic normal - Linear Regression
*   

**Output:**

In [ ]:
#installing packages
!pip install py-climate-health-toolbox

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
#import climahe.climatex as tex
import climatex as tex
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.impute import SimpleImputer

## Loading Data:

*   IAC
*   VCP


### IAC

In [ ]:
IAC = pd.read_csv('/content/drive/Shared drives/Clima&Saúde/Dados/Dados_Clima/IAC/data/processed/IAC_HW1956_2018.csv')
IAC['DATE'] = pd.to_datetime(IAC['DATE'])

NameError: ignored

In [ ]:
#selecting data from the same period as VCP data
df_IAC = IAC[(IAC['YEAR']>=1983)&(IAC['YEAR']<=2018)] #defining database
df_IAC = df_IAC.reset_index()
del df_IAC['index']

In [ ]:
#selecting data correspondent to climatological normal
IAC_climaticNormal = IAC[(IAC['YEAR']>1960)&(IAC['YEAR']<=1990)] #defining climatic normal
IAC_climaticNormal = IAC_climaticNormal.reset_index()
del IAC_climaticNormal['index']

### VCP

In [ ]:
df_VCP = pd.read_csv(r'/home/daniela/Documentos/CliSau/Viracopos_1983_2018.csv')
del df_VCP['Unnamed: 0']
df_VCP['DATE'] = pd.to_datetime(df_VCP['DATE'])

In [ ]:
#completing missing dates on VCP dataframe
databaseVIR=tex.complete_df(databaseVIR)

### Plot IAC vs. VCP 

#### Maximum temperature

In [ ]:
plt.plot(df_IAC['N_AIRTMP_MAX'],df_IAC['N_AIRTMP_MAX'],'c.')
plt.xlabel('IAC: Temperatura máxima (°C)')
plt.ylabel('VCP: Temperatura máxima (°C)')
plt.title('IAC x VCP')

#### Mininum temperature

In [ ]:
plt.plot(df_IAC['N_AIRTMP_MIN'],df_IAC['N_AIRTMP_MIN'],'c.')
plt.xlabel('IAC: Temperatura mínima (°C)')
plt.ylabel('VCP: Temperatura mínima (°C)')
plt.title('IAC x VCP')

## Linear Regression

### Maximum Temperatures

In [ ]:
x = df_IAC['N_AIRTMP_MAX'].values.reshape(-1,1) #transforming column into vector
y = df_VCP['N_AIRTMP_MAX'].values.reshape(-1,1)

In [ ]:
#filling missing data
imputer = SimpleImputer()
x_imputed = imputer.fit_transform(x)
y_imputed = imputer.fit_transform(y)

In [ ]:
x_train = x_imputed
y_train = y_imputed

#performing regression
regressor_max = LinearRegression()  
regressor_max.fit(x_train, y_train)

In [ ]:
#obtaining 
print(regressor_max.intercept_)
print(regressor_max.coef_)

### Minimum Temperatures

In [ ]:
x = df_IAC['N_AIRTMP_MIN'].values.reshape(-1,1) #transforming column into vector
Y = df_VCP['N_AIRTMP_MIN'].values.reshape(-1,1)

In [ ]:
#filling missing data
imputer = SimpleImputer()
X_imputed = imputer.fit_transform(X)
Y_imputed = imputer.fit_transform(Y)

In [ ]:
X_train = X_imputed
Y_train = Y_imputed

#performing regression
regressor_min = LinearRegression()  
regressor_min.fit(X_train, Y_train)

In [ ]:
#obtaining 
print(regressor_min.intercept_)
print(regressor_min.coef_)

## Using linear regression and IAC data (1961-1990) to estimate VCP climatic normal

### Maximum temperatures

In [ ]:
#defining predictor
Z = IAC_climaticNormal['N_AIRTMP_MAX'].values.reshape(-1,1) 

In [ ]:
#filling missing data
Z_imputed = imputer.fit_transform(Z)

In [ ]:
#predicting tmax from VCP using regression
tmax_pred=regressor_max.predict(Z_imputed)

#### Plot Tmax

In [ ]:
plt.plot(Z, tmax_pred, color='red', linewidth=2)
plt.xlabel('IAC: Tmax (°C)')
plt.ylabel('VCP: Tmax (°C)')
plt.title('IAC x VCP: Temperatura máxima')
plt.show()

### Minimum temperatures

In [ ]:
#defining predictor
W = IAC_climaticNormal['N_AIRTMP_MIN'].values.reshape(-1,1) 

In [ ]:
#filling missing data
W_imputed = imputer.fit_transform(W)

In [ ]:
#predicting tmin from VCP using regression
tmin_pred=regressor_min.predict(W_imputed)

#### Plot Tmin

In [ ]:
plt.plot(W, tmin_pred, color='blue', linewidth=2)
plt.xlabel('IAC: Tmin (°C)')
plt.ylabel('VCP: Tmin (°C)')
plt.title('IAC x VCP: Temperatura mínima')
plt.show()

### Constructing dataframe with results for VCP

In [ ]:
date = IAC_climaticNormal['DATE']

#tmax dataframe
tmax = pd.DataFrame(data=tmax_pred, columns=['N_AIRTMP_MAX'])
tmax

In [ ]:
#tmin dataframe
tmin = pd.DataFrame(data=tmin_pred, columns=['N_AIRTMP_MIN'])
tmin

In [ ]:
#concatenating tmax and tmin dataframes
result = pd.concat([date, tmax], axis=1)
result = pd.concat([result,tmin],axis=1)
result

In [ ]:
#saving results
result.to_csv('VCP_nc_6190.csv')

## Computing Heatwaves for VCP weather station

Using function check_HeatWave from climatex library

In [ ]:
#VCP climatic normal
VCP_climaticNormal = result.copy()

#VCP database
databaseVCP = df_VCP.copy()

In [ ]:
#checking function parameters
help(tex.check_HeatWave)

In [ ]:
#computing heatwaves and percentiles threshold
HW_VCP,pct_VCP = tex.check_HeatWave(databaseVCP,'N_AIRTMP_MAX','N_AIRTMP_MIN',
                           VCP_climaticNormal,'N_AIRTMP_MAX','N_AIRTMP_MIN')

In [ ]:
#percentiles threshold
pct_VCP

In [ ]:
#heatwaves dataframe
HW_VCP

In [ ]:
#checking dates with heatwaves (HW = 1)
HW_VCP[HW_VCP['HW']==1]

## Metrics

Using functions wave_metrics and wave_seasonMetrics from climatex library to obtain yearly and seasonal metrics

### Yearly metrics

Function: tex.wave_metrics

In [ ]:
#checking wave_metrics parameters
help(tex.wave_metrics)

In [ ]:
#obtaining yearly metrics
HW_VCP_metrics = tex.wave_metrics(HW_VCP,'HW')
HW_VCP_metrics

### Seasonal metrics

Function: tex.wave_seasonMetrics

In [ ]:
#checking wave_seasonMetrics parameters
help(tex.wave_seasonMetrics)

In [ ]:
#obtaining seasonal metrics
HW_VCP_seasonMetrics = tex.wave_seasonMetrics(HW_VCP,'HW')
HW_VCP_seasonMetrics

## Comparing VCP with IAC

Checking coincident heatwave days

In [ ]:
#only heatwaves dataframe IAC
iac = HW_IAC[(HW_IAC['DATE'].dt.year>=1983)&(HW_IAC['DATE'].dt.year<=2018)]
onlyHW_IAC = iac[iac['HW']==1]

In [ ]:
#only heatwaves dataframe VCP
onlyHW_VCP = HW_VCP[HW_VCP['HW']==1]

In [ ]:
#selecting columns
onlyHW_IAC = onlyHW_IAC[['DATE','HW']]
onlyHW_VCP = onlyHW_VCP[['DATE','HW']]

In [ ]:
#checking number of days with heatwaves IAC
onlyHW_IAC.shape

In [ ]:
#checking number of days with heatwaves VCP
onlyHW_VCP.shape

In [ ]:
#merging IAC and VCP
IAC_VCP = pd.merge(onlyHW_IAC,onlyHW_VCP)

#checking coincident heatwave days
IAC_VCP.shape

## Plot Out 2014 - Fev 2015

### Reading pct

In [ ]:
#reading file with percentiles IAC
pct_IAC = pd.read_csv(r'/home/daniela/Documentos/CliSau/compute_HeatColdWaves/pct_IAC.csv')

In [ ]:
#reading file with percentiles VCP
pct_VCP

### Selecting data: 01.10.2014 a 31.01.2015

In [ ]:
#selecting data - 01/10/2014 a 31/01/2015
start_date = '2014-10-01'
end_date = '2015-01-31'

In [ ]:
#IAC
sel_dataIAC = HW_IAC[(HW_IAC['DATE']>=start_date)&(HW_IAC['DATE']<=end_date)]
merge_IAC = pd.merge(sel_dataIAC, pct_IAC, on='DAY365')
merge_IAC = merge_IAC.sort_values(by='DATE')

In [ ]:
#VCP
sel_dataVIR = HW_VCP[(HW_VCP['DATE']>=start_date)&(expHW_VCP['DATE']<=end_date)]
merge_VCP = pd.merge(sel_dataVCP, pct_VCP, on='DAY365')
merge_VCP = merge_VCP.sort_values(by='DATE')

### Plot

In [ ]:
#CTX90pct N_AIRTMP_MAX
plt.plot(merge_VCP['DATE'], merge_VCP['CTX90pct'],'--',color='orange',label='VCP - CTX90pct')
plt.plot(merge_VCP['DATE'], merge_VCP['N_AIRTMP_MAX'],color='orange',label='VCP - Tmax')
plt.fill_between(merge_VCP['DATE'], merge_VCP['CTX90pct'], merge_VCP['N_AIRTMP_MAX'], where = merge_VCP['CTX90pct'] <= merge_VCP['N_AIRTMP_MAX'], facecolor='orange', interpolate=True)

#CTN90pct N_AIRTMP_MIN
plt.plot(merge_VCP['DATE'], merge_VCP['CTN90pct'],'--',color ='gray',label='VCP - CTN90pct')
plt.plot(merge_VCP['DATE'], merge_VCP['N_AIRTMP_MIN'],color='gray',label='VCP - Tmin')
plt.fill_between(merge_VCP['DATE'], merge_VCP['CTN90pct'], merge_VCP['N_AIRTMP_MIN'], where = merge_VCP['CTN90pct'] <= merge_VCP['N_AIRTMP_MIN'], facecolor='gray', interpolate=True)

plt.title('Weather Stations Comparison: IAC and VCP',fontsize=18)
plt.ylabel('Temperature (°C)',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

fig = plt.gcf()
fig.autofmt_xdate()
plt.grid()
plt.legend(loc=4, prop={'size': 9})
plt.show()